# Накопленный охват по видам спорта
Пример расчета накопленного охвата в разбивке по видам спорта

## Описание задачи и условий расчета
- Период: 01.06.2022 - 31.08.2022
- ЦА: Мужчины 35-59
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Программы: группировка по виду спорта
- Статистики: Duration (тотал), CumReach(000), CumReach(%)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем нужные нам переменные
cats.get_tv_demo_attribute(names=['пол','возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: sex и age

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-06-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА: Мужчины 35-59
basedemo_filter = 'sex = 1 and age >= 35 and age <= 59'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = None

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов (группировок)
slices = [
    'programSportName', #Программа вид спорта
]

# Указываем список статистик для расчета
statistics = ['DurationSum','CumReach000','CumReachPer']

# Задаем условия сортировки: Программа вид спорта (от а до я)
sortings = {'programSportName':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "issueType": "PROGRAM" #Тип события - Программы
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_crosstab = mtask.wait_task(mtask.send_crosstab_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_crosstab))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_cum_reach_by_sports')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)